In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from data import Data
from sklearn.preprocessing import LabelEncoder

In [5]:
df = Data()
encoders=df.encoders
#df[df['state']==0 & df['state']==1].replace(0)

x_train, x, y_train, y = df.dev_data('goal', test_percent=0.01)

In [ ]:
x

In [14]:
y

array(['successful'], dtype=object)

In [ ]:
def best_params(model, params, cv, x_train, y_train):
    grid = GridSearchCV(model, params, cv=cv)
    grid.fit(x_train, y_train)
    best_options = grid.best_params_
    return best_options

def prediction_reg(model, x_train, y_train, x_test, y_test, cv):
    model.fit(x_train, y_train)
    score_val = np.mean(cross_val_score(model, x_train, y_train, cv=cv))
    y_pred = model.predict(x_test)
    score_acc = None
    score = model.score(x_train, y_train)
    return score_val, score_acc, score

def normalize_data(x_train, x_test):
    scaler = MinMaxScaler()
    scaler.fit(x_train)
    x_train_norm = scaler.transform(x_train)
    x_test_norm = scaler.transform(x_test)
    return x_train_norm, x_test_norm




# DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr_params = {"criterion": ["mse", "friedman_mse", "mae"],
              "splitter": ["best", "random"]}
dtr_best_params = best_params(dtr, dtr_params, 3, x_train, y_train)
dtr_best = DecisionTreeRegressor(**dtr_best_params)
dtr_result = prediction_reg(dtr_best, x_train, y_train, x_test, y_test, cv=3)


# KNeighborsRegressor
knr = KNeighborsRegressor()
knr_params = {'weights': ["uniform", "distance"],
              'p': [1, 2],
              'n_neighbors': np.arange(1, 40)}
knc_best_params = best_params(knr, knr_params, 3, x_train_norm, y_train)
knr_best = KNeighborsClassifier(**knc_best_params)
knr_result = prediction(knr_best, x_train_norm, y_train, x_test_norm, y_test, cv=3)

# RadiusNeighborsRegressor
rnr = RadiusNeighborsRegressor()
rnr_params = {}
rnr_best_params = best_params(rnr, rnr_params, 3, x_train_norm, y_train)
rnr_best = RadiusNeighborsRegressor(**rnr_best_params)
rnr_result = prediction_reg(rnr_best, x_train_norm, y_train, x_test_norm, y_test, cv=3)



# RandomForestRegressor
rfr = RandomForestRegressor()
rfr_params = {"n_estimators": np.arange(10, 15, 1),
              "min_samples_split": np.arange(2, 5, 1),
              "min_samples_leaf": np.arange(1, 5, 1),
              "criterion": ["mse", "mae"]}
rfr_best_params = best_params(rfr, rfr_params, 3, x_train, y_train)
rfr_best = RandomForestRegressor(**rfr_best_params)
rfr_result = prediction_reg(rfr_best, x_train, y_train, x_test, y_test, cv=3)



# SVM
svm_model = svm.SVC()
svm_params = {'kernel': ['linear'],
              'gamma': ['scale','auto']}
svm_best_params = best_params(svm_model, svm_params, 2, x_train, y_train)
svm_best = svm.SVC(**svm_best_params)
svm_result = prediction(svm_best, x_train, y_train, x_test, y_test, cv=3)



# Bayes
bayes_model = GaussianNB()
bayes_params = {'priors': None
                #'var_smoothing': [1e-9]
                }
bayes_best_params = best_params(bayes_model, bayes_params, 2, x_train, y_train)
bayes_best = svm.SVC(**bayes_best_params)
bayes_result = prediction(bayes_best, x_train, y_train, x_test, y_test, cv=3)




all_results = [
               ['DecisionTreeRegressor', *dtr_result],
               ['KNeighborsRegressor',*knr_result],
               ['RadiusNeighborsRegressor',*rnr_result],
               ['RandomForestRegressor', *rfr_result],
               ['SVM', *svm_result],
               ['Byess', *bayes_result]
               ]


result_df = pd.DataFrame(all_results, columns=["model", "mean_val_score", "accuracy_score", "score"])
print(result_df)